# Intro to Autogen and RAG
Notebook focuses on creating...

## Import ChromaDB and configure local store

In [1]:
import chromadb
from chromadb.config import Settings

# Create file-based vector store
# NOTE: turned off telemetry as causing error
client_settings = Settings(
    anonymized_telemetry=False,
)
client = chromadb.PersistentClient(path="./chroma_db", settings=client_settings)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [2]:
# Create (or load) a collection
collection = client.get_or_create_collection(
    name="rag_collection",
    metadata={"hnsw:space": "cosine"}
)

Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [3]:
# Add some sample docs (can replace later with PDFs, scraped text, embeddings)
docs = [
    "AutoGen is a framework for building multi-agent systmes.",
    "RAG stands for Retrieval-Augmented Generation.",
    "Vector databases store embeddings for semantic search."
]

for i, doc in enumerate(docs):
    collection.add(
        documents=[doc],
        ids=[str(i)]
    )

Insert of existing embedding ID: 0
Add of existing embedding ID: 0
Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given
Insert of existing embedding ID: 1
Add of existing embedding ID: 1
Insert of existing embedding ID: 2
Add of existing embedding ID: 2


In [4]:
# Define simple retriever helper
def retrieve(query, k=3):
    results = collection.query(query_texts=[query], n_results=k)
    return results["documents"][0]

## Create AutoGen Agents
### Load API Key and Configure LLM


In [30]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os
import asyncio

# Load environment varaibles
load_dotenv()

True

In [16]:
# Create LLM configuration
if os.getenv("OPENAI_API_KEY"):
    api_key=os.getenv("OPENAI_API_KEY")
else:
    print("Unable to load OPENAI_API_KEY")

model_client = OpenAIChatCompletionClient(model = 'gpt-4o', api_key=api_key)

### Create Agents

In [ ]:
# Create AssistantAGent
assistant = AssistantAgent(
    name="assistant",
    model_client=model_client
)

# Create UserProxyAgent
user = UserProxyAgent(
    name="user",
    input_func="NEVER"    # agent will run without needing to pause for human input
)


## Create RAG wrapper for AutoGen assistant
Combines
- user's query
- retrieved context
- structured prompt
- AutoGen assistant agent

In [32]:
async def rag_answer(query:str):
    # Retrieve context
    context_docs = retrieve(query)
    context = "\n\n".join(context_docs)

    # Build agumented prompt
    prompt = f"""
    Use the following context to answer the question.

    Context:
    {context}

    Question:
    {query}
    """

    # Send to Autogen Assistant
    result = await assistant.run(task=prompt)
    
    return result


## Test RAG system

In [34]:
result = await rag_answer("What is AutoGen?")
print(result.messages[-1].content)

/Users/briannabesinaiz/Documents/04_Projects/GITHUB/repos/python_portfolio/python-portfolio/autogen_lab/autogen_venv/lib/python3.11/site-packages/pydantic/_internal/_typing_extra.py:492: RuntimeWarning: coroutine 'BaseChatAgent.run' was never awaited
  return _eval_type(value, globalns, localns, type_params)


AutoGen is a framework for building multi-agent systems. TERMINATE
